<h1>Para começar, importa-se as libs necessárias para os procedimentos de data cleaning juntamente com o dataframe de reviews</h1>

In [23]:
import pandas as pd
from os.path import abspath
import os

# Remove o arquivo final caso ele exista no diretório dos databases
try:
    pd.read_csv(abspath('./data/data_cat.gen.csv'))
    pd.read_csv(abspath('./data/data_bin.gen.csv'))
except:
    print("Não existe os arquivos gen.csv")
else:
    os.remove(abspath('./data/data_cat.gen.csv'))
    os.remove(abspath('./data/data_bin.gen.csv'))

reviews = pd.read_csv(abspath('./data/olist_order_reviews_dataset.csv'))


<h2>Cria-se função para obter demais informações dos dados</h2>


In [24]:
def dfOverview(dataframe):
    """This function will return the overview of the dataframe"""

    print("O tamanho do dataframe é (linha, coluna): {}".format(dataframe.shape))
    print("**"*30)
    print("Número total de valores nulos: \n ", dataframe.isnull().sum())
    print("**"*30)

    return dataframe.head()


<h2>Por exemplo, pode-se analisar os dados da base de reviews:</h2>

In [25]:
dfOverview(reviews)

O tamanho do dataframe é (linha, coluna): (100000, 7)
************************************************************
Número total de valores nulos: 
  review_id                      0
order_id                       0
review_score                   0
review_comment_title       88285
review_comment_message     58247
review_creation_date           0
review_answer_timestamp        0
dtype: int64
************************************************************


,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


<h2><strong>Observações<strong></h2>


<ul>
<li>Valores nulos se fazem presentes no dataframe de reviews;</li>
</ul>

<p>Separa-se dois datasets contendo informações dos comentários dos clientes (review_comment_message) e da sua nota de avaliação (review_score). Para o data_bin tem-se comentários associados categoricamente (0 e 1) a partir de intervalor de notas (0,2] e (2, 5] e no data_cat tem-se apenas os comentários e os reviews numéricos de 0 a 5. Ambas as bases são removidos valores nulos</p>

In [26]:
def clean_data(data, column_text='review_comment_message',
               column_score='review_score',
               points_cut=[0, 2, 5],
               classes=[0, 1]):

    df_bin = data
    df_bin = df_bin.dropna(subset=[column_text])
    df_bin['label'] = pd.cut(df_bin[column_score],
                             bins=points_cut, labels=classes)
    df_bin = df_bin.rename(columns={column_text: 'text'})
    df_bin = df_bin[['text', 'label']]
    print('data_bin ==> A quantidade de linhas totais apos remoção de nulos e adequação: ',
          df_bin.shape[0])
          
    df_cat = data
    df_cat = df_cat.dropna(subset=[column_text])
    df_cat = df_cat.rename(
        columns={column_text: 'text', column_score: 'label'})
    df_cat = df_cat[['text', 'label']]
    print('data_cat ==> A quantidade de linhas totais apos remoção de nulos e adequação: ',
          df_cat.shape[0])
    print('Havia cerca de:', round(
        (data.shape[0] - df_cat.shape[0])*100/data.shape[0], 3),"% de dados nulos no dataset")
    return df_bin, df_cat


data_bin, data_cat = clean_data(reviews)
data_cat.isnull().sum()
data_bin.isnull().sum()


data_bin ==> A quantidade de linhas totais apos remoção de nulos e adequação:  41753
data_cat ==> A quantidade de linhas totais apos remoção de nulos e adequação:  41753
Havia cerca de: 58.247 % de dados nulos no dataset


/home/carlos/carlos/ECD/python/venv/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


text     0
label    0
dtype: int64

<h2>Por fim, salvamos os dados em dois novos arquivos .csv</h2>

In [27]:
data_bin.to_csv(abspath("data/data_bin.gen.csv"), index=False)
data_cat.to_csv(abspath("data/data_cat.gen.csv"), index=False)
